In [73]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load Raw Data

In [74]:
# Step 2: Load Raw Data
df = pd.read_csv("raw_data.csv")
df.head()

,DateTime,Junction,Vehicles,ID,Temperature,Precipitation,Humidity,WindSpeed,Date,Event_Type,Event_Name,Event_Flag
0,2015-11-01 00:00:00,1,15,20151101001,23.2,0.0,67,11.4,2015-11-01,No Event,No Event,0
1,2015-11-01 01:00:00,1,13,20151101011,22.6,0.0,70,10.7,2015-11-01,No Event,No Event,0
2,2015-11-01 02:00:00,1,10,20151101021,22.0,0.0,73,10.0,2015-11-01,No Event,No Event,0
3,2015-11-01 03:00:00,1,7,20151101031,21.5,0.0,75,8.2,2015-11-01,No Event,No Event,0
4,2015-11-01 04:00:00,1,9,20151101041,21.0,0.0,77,8.4,2015-11-01,No Event,No Event,0


In [75]:
# check shape of data
df.shape

(48120, 12)

In [76]:
# 48120 rows and 12 columns

In [77]:
# print columns name
df.columns

Index(['DateTime', 'Junction', 'Vehicles', 'ID', 'Temperature',
       'Precipitation', 'Humidity', 'WindSpeed', 'Date', 'Event_Type',
       'Event_Name', 'Event_Flag'],
      dtype='str')

In [78]:
# 1) DateTime
# Date and time of traffic record
# Used to extract hour, day, month
# Important for time-series forecasting

# 2) Junction
# ID of road junction/intersection
# Helps analyze traffic per location

# 3) Vehicles
# Number of vehicles recorded
# Target variable (what we predict)

# 4) ID
# Unique row identifier
# Not useful for prediction
# Can be removed

# ️5) Temperature
# Temperature at that time (°C)
# May influence traffic flow

# ️6) Precipitation
# Rainfall amount
# Heavy rain may increase congestion

# ️7) Humidity
# Air moisture percentage
# Minor effect on traffic

# ️8) WindSpeed
# Wind speed at that time
# Usually small impact

# ️9) Date
# Date without time
# Can be removed if DateTime is used

# 10) Event_Type
# Type of event (Festival, Holiday, etc.)
# Events increase traffic

# 11) Event_Name
# Specific event name
# Can be encoded if needed

# 12) Event_Flag
# Binary indicator (1 = event, 0 = no event)


In [79]:
# check information of data
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 48120 entries, 0 to 48119
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   DateTime       48120 non-null  str    
 1   Junction       48120 non-null  int64  
 2   Vehicles       48120 non-null  int64  
 3   ID             48120 non-null  int64  
 4   Temperature    48120 non-null  float64
 5   Precipitation  48120 non-null  float64
 6   Humidity       48120 non-null  int64  
 7   WindSpeed      48120 non-null  float64
 8   Date           48120 non-null  str    
 9   Event_Type     48120 non-null  str    
 10  Event_Name     48120 non-null  str    
 11  Event_Flag     48120 non-null  int64  
dtypes: float64(3), int64(5), str(4)
memory usage: 6.5 MB


In [80]:
# observation:-
# from above it is observe that DateTime column datatype is str it is not datetime

# Clean Data:

In [81]:
# Step 3: Convert Datetime column datatype into DateTime
df['DateTime'] = pd.to_datetime(df['DateTime'])

In [82]:
print(df['DateTime'].dtype)

datetime64[us]


In [83]:
# Step 4: Check Missing Values
df.isnull().sum()

DateTime         0
Junction         0
Vehicles         0
ID               0
Temperature      0
Precipitation    0
Humidity         0
WindSpeed        0
Date             0
Event_Type       0
Event_Name       0
Event_Flag       0
dtype: int64

In [84]:
# observation:-
# there is no any null value present in dataset

In [85]:
# Step 5: Handle Missing Values - if any missing value present then it will handle
# Fill numerical columns with mean
num_cols = ['Vehicles', 'Temperature', 'Precipitation',
            'Humidity', 'WindSpeed']

for col in num_cols:
    df[col] = df[col].fillna(df[col].mean())

# Fill categorical columns with mode (if any missing)
cat_cols = ['Event_Type', 'Event_Name']
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

print("\nMissing Values After Handling:\n", df.isnull().sum())


Missing Values After Handling:
 DateTime         0
Junction         0
Vehicles         0
ID               0
Temperature      0
Precipitation    0
Humidity         0
WindSpeed        0
Date             0
Event_Type       0
Event_Name       0
Event_Flag       0
dtype: int64


In [86]:
df.shape

(48120, 12)

In [87]:
# Step 6: check duplicate values
df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
48115    False
48116    False
48117    False
48118    False
48119    False
Length: 48120, dtype: bool

In [88]:
# Remove Duplicates if any present
df = df.drop_duplicates()

In [89]:
print("Shape After Removing Duplicates:", df.shape)

Shape After Removing Duplicates: (48120, 12)


In [90]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 48120 entries, 0 to 48119
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   DateTime       48120 non-null  datetime64[us]
 1   Junction       48120 non-null  int64         
 2   Vehicles       48120 non-null  int64         
 3   ID             48120 non-null  int64         
 4   Temperature    48120 non-null  float64       
 5   Precipitation  48120 non-null  float64       
 6   Humidity       48120 non-null  int64         
 7   WindSpeed      48120 non-null  float64       
 8   Date           48120 non-null  str           
 9   Event_Type     48120 non-null  str           
 10  Event_Name     48120 non-null  str           
 11  Event_Flag     48120 non-null  int64         
dtypes: datetime64[us](1), float64(3), int64(5), str(3)
memory usage: 5.6 MB


# Aggregate Traffic Data Hourly

In [91]:
# Forecast hourly traffic volume at different junctions. But raw data may be:
# Minute-wise
# Multiple records per hour
# Irregular timestamps

# Machine learning models need:Clean, Structured, Consistent hourly intervals
# We aggregate because:
# To Convert Raw Data → Hourly Traffic
# If 20 vehicle entries exist in one hour, we combine them.

In [92]:
df['DateTime'] = pd.to_datetime(df['DateTime'])  # ensure datetime
df = df.set_index('DateTime')                    # set as index

In [93]:
# Aggregate hourly per junction
df_hourly = df.groupby('Junction').resample('h').agg({
    'Vehicles': 'sum',          # Total vehicles per hour
    'Temperature': 'mean',      # Average temperature
    'Precipitation': 'mean',    # Average precipitation
    'Humidity': 'mean',         # Average humidity
    'WindSpeed': 'mean',        # Average wind speed
    'Event_Flag': 'max'         # If any event occurred in that hour
}).reset_index()

In [94]:
print("Hourly Aggregated Data Shape:", df_hourly.shape)

Hourly Aggregated Data Shape: (48120, 8)


In [95]:
print(df_hourly.head())

   Junction            DateTime  Vehicles  Temperature  Precipitation  \
0         1 2015-11-01 00:00:00        15         23.2            0.0   
1         1 2015-11-01 01:00:00        13         22.6            0.0   
2         1 2015-11-01 02:00:00        10         22.0            0.0   
3         1 2015-11-01 03:00:00         7         21.5            0.0   
4         1 2015-11-01 04:00:00         9         21.0            0.0   

   Humidity  WindSpeed  Event_Flag  
0      67.0       11.4           0  
1      70.0       10.7           0  
2      73.0       10.0           0  
3      75.0        8.2           0  
4      77.0        8.4           0  


In [96]:

# Feature Engineering 

df_hourly['Hour'] = df_hourly['DateTime'].dt.hour
df_hourly['Day'] = df_hourly['DateTime'].dt.day
df_hourly['Month'] = df_hourly['DateTime'].dt.month
df_hourly['DayOfWeek'] = df_hourly['DateTime'].dt.dayofweek
df_hourly['IsWeekend'] = df_hourly['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

In [97]:
# STEP 6: Standardization

scaler = StandardScaler()

scale_cols = ['Temperature', 'Precipitation',
              'Humidity', 'WindSpeed']

df_hourly[scale_cols] = scaler.fit_transform(df_hourly[scale_cols])

print("\nFinal Processed Data:")
print(df_hourly.head())


Final Processed Data:
   Junction            DateTime  Vehicles  Temperature  Precipitation  \
0         1 2015-11-01 00:00:00        15    -0.487137      -0.149178   
1         1 2015-11-01 01:00:00        13    -0.596649      -0.149178   
2         1 2015-11-01 02:00:00        10    -0.706161      -0.149178   
3         1 2015-11-01 03:00:00         7    -0.797422      -0.149178   
4         1 2015-11-01 04:00:00         9    -0.888682      -0.149178   

   Humidity  WindSpeed  Event_Flag  Hour  Day  Month  DayOfWeek  IsWeekend  
0  0.631414  -0.197765           0     0    1     11          6          1  
1  0.752838  -0.318525           0     1    1     11          6          1  
2  0.874261  -0.439286           0     2    1     11          6          1  
3  0.955210  -0.749814           0     3    1     11          6          1  
4  1.036159  -0.715311           0     4    1     11          6          1  


In [98]:
# FINAL DATA INFO
print("\nFinal Dataset Info:")
print(df_hourly.info())


Final Dataset Info:
<class 'pandas.DataFrame'>
RangeIndex: 48120 entries, 0 to 48119
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Junction       48120 non-null  int64         
 1   DateTime       48120 non-null  datetime64[us]
 2   Vehicles       48120 non-null  int64         
 3   Temperature    48120 non-null  float64       
 4   Precipitation  48120 non-null  float64       
 5   Humidity       48120 non-null  float64       
 6   WindSpeed      48120 non-null  float64       
 7   Event_Flag     48120 non-null  int64         
 8   Hour           48120 non-null  int32         
 9   Day            48120 non-null  int32         
 10  Month          48120 non-null  int32         
 11  DayOfWeek      48120 non-null  int32         
 12  IsWeekend      48120 non-null  int64         
dtypes: datetime64[us](1), float64(4), int32(4), int64(4)
memory usage: 4.0 MB
None
